In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from datasets import load_dataset
import pandas as pd
from readability import Readability

device = 'cuda:3' # if you have a GPU

2023-03-01 04:25:19.009316: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 04:25:19.009339: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl").to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
wizard = pd.read_json("/home/prasann/miniconda3/envs/rewardenv/lib/python3.8/site-packages/data/wizard_of_wikipedia/test_random_split.json")

In [106]:
def construct_prompt(row):
    template = \
"""Knowledge: {knowledge:s}
    
The System summarizes the Knowledge, and responds in an engaging conversation with the user. It asks the user for more to continue the conversation.
 
System: Hello! What do you want to talk about today?
User: {start:s}
System:"""
    know = ""
    for c in row['chosen_topic_passage']:
        know = know + c + " "
    inp = template.format(topic=row['chosen_topic'], start=row['dialog'][0]['text'], knowledge=know)
    return inp

def gen_row(rw, tok, mod, greedy=False, log=False):
    input_text = construct_prompt(rw)
    
    #print(input_text)
    input_ids = tok(input_text, return_tensors="pt").input_ids.to(device)
    if greedy:
        outputs = mod.generate(input_ids, min_new_tokens=20, max_new_tokens=200)
        outs = [tok.decode(outputs[0], skip_special_tokens=True)]
    else: 
        outputs = mod.generate(input_ids, min_new_tokens=20, max_new_tokens=200, do_sample=True, top_p=.9, temperature=.9, num_return_sequences=10)
        outs = [tok.decode(o, skip_special_tokens=True) for o in outputs]
    if log:
        print(input_text+"\n"+str(outs))
    return rw['dialog'][0]['text'], outs

In [107]:
inp, outs = gen_row(wizard.iloc[6], tokenizer, model, False)

In [104]:
steamtok = T5Tokenizer.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl')
steamshp = T5ForConditionalGeneration.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl').to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [126]:
def get_reward_single(inpdict):
    template = "POST: {context:s} \n\nRESPONSE A:{hyp:s} \n\nRESPONSE B: .\n\n Which response is better? RESPONSE "
    inp = template.format(context=inpdict['context'], hyp=inpdict['hyp'])
    x = steamtok([inp], return_tensors='pt').input_ids.to(device)
    outputs = steamshp.generate(x, return_dict_in_generate=True, output_scores=True, max_new_tokens=1)
    return torch.exp(outputs.scores[0][:, 71]) / torch.exp(outputs.scores[0][:,:]).sum(axis=1).item() # index 71 corresponds to the token for 'A'

def score_dialogue(start, hyp):
    cont = start+" Does anyone have any relevant thoughts or information? More details would be better."
    sco = float(get_reward_single({"context":cont , "hyp":hyp}))
    return sco

def comp_rerank_greedy(row):
    inp, outs = gen_row(row, tokenizer, model, False) # do sampling for rerank
    _, greedy_out = gen_row(row, tokenizer, model, True) # do greedy 
    scos = [score_dialogue(inp, o) for o in outs]
    greedysco = score_dialogue(inp, greedy_out[0])
    # return list of stuff to put in dict 
    return pd.DataFrame(list([{'gout':greedy_out[0], 'gsco':greedysco, 'inp':inp, 'hyp':outs[i], 'sco':scos[i]} for i in range(len(scos))]))
    


In [133]:
def get_many_exouts(total):
    resdfs = []
    for i in range(total):
        print(i)
        resdfs.append(comp_rerank_greedy(wizard.iloc[i]))
    return resdfs

In [172]:
def print_wiz_data(ind):
    pmpt = construct_prompt(wizard.iloc[ind])
    print(pmpt)
    outs = check_df.iloc[ind*10:(ind+1)*10]
    print("Greedy Output: \n", outs['gsco'].iloc[0], " ", outs['gout'].iloc[0], "\n")
    print("Sampling Output: \n")
    for o in range(10):
        print( outs['sco'].iloc[o], " ",outs['hyp'].iloc[o], "\n")
    

In [186]:
print_wiz_data(16)

Knowledge: Science fiction (often shortened to SF or sci-fi) is a genre of speculative fiction, typically dealing with imaginative concepts such as futuristic science and technology, space travel, time travel, faster than light travel, parallel universes, and extraterrestrial life. Science fiction often explores the potential consequences of scientific and other innovations, and has been called a "literature of ideas". It usually avoids the supernatural, unlike the related genre of fantasy. Historically, science-fiction stories have had a grounding in actual science, but now this is only expected of hard science fiction. Science fiction is difficult to define, as it includes a wide range of subgenres and themes. Hugo Gernsback, who suggested the term "scientifiction" for his "Amazing Stories" magazine, wrote: "By 'scientifiction' I mean the Jules Verne, H. G. Wells and Edgar Allan Poe type of story—a charming romance intermingled with scientific fact and prophetic vision... Not only do

In [ ]:
check_outs = get_many_exouts(50)

In [135]:
check_df = pd.concat(check_outs)

In [187]:
check_df['sco'].mean()

0.7093579197824001

In [141]:
check_df

,gout,gsco,inp,hyp,sco
0,Ok. Here are some facts about blue. Blue is on...,0.807007,Blue is my favorite primary color.,Happy to help! Let me just confirm that the co...,0.624099
1,Ok. Here are some facts about blue. Blue is on...,0.807007,Blue is my favorite primary color.,What would you like to learn about? You can fi...,0.505249
2,Ok. Here are some facts about blue. Blue is on...,0.807007,Blue is my favorite primary color.,What is blue? rnrnrnrn,0.523563
3,Ok. Here are some facts about blue. Blue is on...,0.807007,Blue is my favorite primary color.,"Well, let me do some research. Blue is one of ...",0.482060
4,Ok. Here are some facts about blue. Blue is on...,0.807007,Blue is my favorite primary color.,"Yes, I can help you with that. Let me look som...",0.533534
...,...,...,...,...,...
5,"Oh, I see. Walmart is a very large company. It...",0.715410,Walmart is the only big box store in my town s...,"Yeah, but it's a chain, so you are able to fin...",0.514161
6,"Oh, I see. Walmart is a very large company. It...",0.715410,Walmart is the only big box store in my town s...,Do you have a specific type of Walmart? I can ...,0.413809
7,"Oh, I see. Walmart is a very large company. It...",0.715410,Walmart is the only big box store in my town s...,Oh! Walmart is one of the biggest retailers in...,0.825878
8,"Oh, I see. Walmart is a very large company. It...",0.715410,Walmart is the only big box store in my town s...,What do you do there? Do you want me to do som...,0.251252


In [154]:
rer_met('sco', 'sco', check_df, False)

0.895716643333435

In [145]:
def rer_met(rer, tgt, df, get_df=True):
    N=1
    fsort = df.sort_values(by=['gout', rer], ascending=[True, False]).groupby('gout', as_index=False).nth[:N]
    if get_df:
        return fsort
    else:
        return fsort[tgt].mean()

In [130]:
list(zip(list(exout['hyp']), list(exout['sco'])))

[('OK. Here are some basic facts about blue. Blue is one of the three primary colors of pigments in painting and traditional color theory, as well as in the RGB color model. It lies between violet and green on the spectrum of visible light. The eye perceives blue when observing light with a dominant wavelength between approximately 450 and 495 nanometres. Most blues contain a slight mixture of other colors; azure contains some green, while ultramarine contains some violet. The clear daytime sky and the deep sea appear blue because of an optical effect known as Rayleigh scattering. An optical effect called Tyndall scattering explains blue eyes. Distant objects appear more blue because of another optical effect called atmospheric perspective. In the eighth century Chinese artists used cobalt blue to colour fine blue and white porcelain. In the Middle Ages, European artists used it in the windows of Cathedrals. Europeans wore clothing coloured with the',
  0.8639145493507385),
 ('How is y

In [119]:
scos = []
for o in outs:
    scos.append(score_dialogue(inp, o))

In [120]:
print(list(zip(scos, outs)))

[(0.9600575566291809, 'Adolph Coors and Jacob Schueler from Prussia. In 1873, German immigrants Adolph Coors and Jacob Schueler from Prussia emigrated to the United States and established a brewery in Golden, Colorado, after buying a recipe for a Pilsner-style beer from a Czech immigrant William Silhan.'), (0.7504631280899048, 'Sure. Which question are you asking me?. For example, "who founded Coors Brewing Company?".'), (0.9470096230506897, 'Adolph Coors and Jacob Schueler. They founded the Coors Brewing Company in Colorado.'), (0.9588615894317627, "Sure. I can help you with that. Let's do this. You want to talk about Adolph Coors and Jacob Schueler."), (0.9791781902313232, 'Adolph Coors and Jacob Schueler from Prussia emigrated to the United States and established a brewery in Golden, Colorado, after buying a recipe for a Pilsner-style beer from a Czech immigrant William Silhan.'), (0.9568148851394653, 'Adolph Coors and Jacob Schueler. Adolph Coors and Jacob Schueler from Prussia emi